In [ ]:
!pip install keras_preprocessing

In [ ]:
!pip install dill

In [ ]:
import functools
import operator
import os
import cv2
import time
import numpy as np
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
def vader_sentiment_analysis(text):
  analyzer = SentimentIntensityAnalyzer()
  sentiment_scores = analyzer.polarity_scores(text)
  compound_score = sentiment_scores['compound']

  if compound_score >= 0.05:
    return "POSITIVE"
  elif compound_score <= -0.05:
    return "NEGATIVE"
  else:
    return "NEUTRAL"

In [ ]:
%run "/content/drive/MyDrive/Colab Notebooks/preprocessing2.ipynb"

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
%run "/content/drive/MyDrive/Colab Notebooks/config.ipynb"

0.005
400


In [ ]:
%run "/content/drive/MyDrive/Colab Notebooks/inference_model.ipynb"

0.005
400
50
400
/content/drive/MyDrive/N_model_lr5e-3
30


In [ ]:
%run "/content/drive/MyDrive/Colab Notebooks/vid_to_frames.ipynb"

Video FPS: 0
Total Frames: 0
frame extraction time: 0.06521248817443848 seconds
50 frames randomly selected and saved successfully!




In [ ]:
class VideoDescriptionRealTime(object):
  """
  Initialize the parameters for the model
  """
  def __init__(self):
    self.latent_dim = latent_dim
    self.num_encoder_tokens = num_encoder_tokens
    self.num_decoder_tokens = num_decoder_tokens
    self.time_steps_encoder = time_steps_encoder
    self.max_probability = max_probability

    self.tokenizer, self.inf_encoder_model, self.inf_decoder_model = inference_model()
    self.save_model_path = save_model_path
    self.test_path = test_path
    self.search_type = search_type
    self.num = 0

  def greedy_search(self, loaded_array):
    """
    :param f: the loaded numpy array after creating videos to frames and extracting features
    :return : the final sentence which has been predicted greedily
    """
    inv_map = self.index_to_word()
    states_value = self.inf_encoder_model.predict(loaded_array.reshape(-1, 50, 4096))
    target_seq = np.zeros((1, 1, 400))
    final_sentence = ''
    target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
    for i in range(15):
      output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
      states_value = [h, c]
      output_tokens = output_tokens.reshape(self.num_decoder_tokens)
      y_hat = np.argmax(output_tokens)
      if y_hat == 0:
        continue
      if inv_map[y_hat] is None:
        break
      if inv_map[y_hat] == 'eos':
        break
      else:
        final_sentence = final_sentence + inv_map[y_hat] + ' '
        target_seq = np.zeros((1, 1, 400))
        target_seq[0, 0, y_hat] = 1

    final_sentence = final_sentence.strip().capitalize()

    final_sentence = final_sentence.rstrip() + '.'

    return final_sentence

  def index_to_word(self):
    index_to_word = {value: key for key, value in self.tokenizer.word_index.items()}
    return index_to_word

  def get_video(self):
    video_folder = '/content/drive/MyDrive/realtimeVideo'

    video_list = os.listdir(video_folder)

    video_files = [video for video in video_list if video.endswith('.mov')]

    video_name = video_files[self.num] # name with extension
    video_id = os.path.splitext(video_name)[0] # name without extension

    save_path = "/content/drive/MyDrive/VideoFrame"

    model = load_vgg16()

    video_path = os.path.join(video_folder, video_name)
    video_to_frames(video_path, save_path)
    frames_save_path = os.path.join(save_path, video_id)

    start_time = time.time()

    features = feature_extraction(frames_save_path, model)

    end_time = time.time()
    difference = end_time - start_time
    print(f"feature extraction time: {difference} seconds")
    print(features.shape)
    print('\n')

    if self.num < len(video_files):
      self.num += 1
    else:
      self.num = 0

    return features, video_name

  def test(self):
    X_test, filename = self.get_video()

    if self.search_type == 'greedy':
      start_time = time.time()
      sentence_predicted = self.greedy_search(X_test.reshape((-1, 50, 4096)))
      end_time = time.time()
      difference = end_time - start_time
      print(f"caption prediction time: {difference} seconds")
      print('\n')

    self.max_probability = -1

    return sentence_predicted, filename

In [ ]:
if __name__ == "__main__":
  video_to_text = VideoDescriptionRealTime()

  while True:
    start = time.time()
    video_caption, file = video_to_text.test()
    end = time.time()
    print('Generated caption:', video_caption)

    print('\n')
    print('It takes {:.2f} seconds to generate caption.'.format(end-start))
    print('\n')

    # Perform sentiment analysis
    result = vader_sentiment_analysis(video_caption)

    # Display sentiment analysis results
    print(result)
    print('\n')

Video FPS: 30
Total Frames: 252
frame extraction time: 4.755063772201538 seconds
50 frames randomly selected and saved successfully!


1/1 [==============================] - 32s 32s/step


feature extraction time: 41.68971657752991 seconds
(50, 4096)


1/1 [==============================] - 0s 29ms/step
caption prediction time: 1.6939358711242676 seconds


Generated caption: A patient is taking rest.


It takes 57.95 seconds to generate caption.


NEUTRAL


Video FPS: 30
Total Frames: 249
frame extraction time: 5.830034017562866 seconds
50 frames randomly selected and saved successfully!


1/1 [==============================] - 32s 32s/step


feature extraction time: 32.655274868011475 seconds
(50, 4096)


1/1 [==============================] - 0s 26ms/step
caption prediction time: 0.8878083229064941 seconds


Generated caption: A patient is drinking water.


It takes 43.83 seconds to generate caption.


NEUTRAL


Video FPS: 30
Total Frames: 129
frame extraction time: 17.27852678298950